In [1]:
!pip install uv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 46.9 MB/s eta 0:00:00


In [ ]:
!uv pip install haystack-ai
!uv pip install sentence-transformers

In [22]:
from haystack import Document
from haystack.components.writers import DocumentWriter
from haystack.document_stores.in_memory import InMemoryDocumentStore
#from haystack.document_stores import InMemoryDocumentStore
from haystack.components.converters import TextFileToDocument
from haystack.components.preprocessors import DocumentSplitter,DocumentCleaner
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack import Pipeline
from haystack.components.joiners import DocumentJoiner



document_store = InMemoryDocumentStore( )
text_file_converter = TextFileToDocument( )
document_cleaner = DocumentCleaner( )
#document_splitter = DocumentSplitter(split_by )
#document_splitter = DocumentSplitter(split_by = "word",split_length = 150,split_overlap=50 )
document_embedder = SentenceTransformersDocumentEmbedder (model="sentence-transformers/all-MiniLM-L6-v2")
document_writer = DocumentWriter(document_store)
document_joiner = DocumentJoiner( )

preprocessing_pipeline = Pipeline( )
#preprocessing_pipeline.add_component(instance=text_file_converter,name="text_file_converter" )
#preprocessing_pipeline.add_component(instance=document_joiner,name="document_joiner" )
#preprocessing_pipeline.add_component(instance=document_cleaner,name="document_cleaner" )
#preprocessing_pipeline.add_component(instance=document_splitter,name="document_splitter" )
preprocessing_pipeline.add_component(instance=document_embedder,name="document_embedder" )
preprocessing_pipeline.add_component(instance=document_writer,name="document_writer" )

#preprocessing_pipeline.connect("text_file_converter","document_joiner"  )
#preprocessing_pipeline.connect("document_joiner","document_cleaner"  )
#preprocessing_pipeline.connect("document_cleaner","document_embedder"  )
#preprocessing_pipeline.connect("document_splitter","document_embedder"  )
preprocessing_pipeline.connect("document_embedder","document_writer"  )




🚅 Components
  - document_embedder: SentenceTransformersDocumentEmbedder
  - document_writer: DocumentWriter
🛤️ Connections
  - document_embedder.documents -> document_writer.documents (List[Document])

In [8]:
import json
import os
import re
from haystack import Document


def load_scraped_docs(docs_dir):
    """Load scraped documents from text files"""
    documents = []

    for filename in os.listdir(docs_dir):
        if filename.endswith('.txt'):
            file_path = os.path.join(docs_dir, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()

                # Split by the separator
                pages = content.split('=' * 80)

                for page in pages:
                    if not page.strip():
                        continue

                    # Extract metadata and content
                    url_match = re.search(r'url: (.*?)$', page, re.MULTILINE)
                    title_match = re.search(r'title: (.*?)$', page, re.MULTILINE)
                    content_match = re.search(r'content: ({.*?})$', page, re.MULTILINE | re.DOTALL)

                    if url_match and title_match and content_match:
                        url = url_match.group(1).strip()
                        title = title_match.group(1).strip()

                        try:
                            content_json = json.loads(content_match.group(1))

                            # Combine text and code blocks
                            text_content = "\n\n".join(content_json.get('text', []))

                            # Format code blocks with markdown
                            code_blocks = []
                            for block in content_json.get('code_blocks', []):
                                lang = block.get('language', '')
                                code = block.get('code', '')
                                code_blocks.append(f"```{lang}\n{code}\n```")

                            full_content = text_content
                            if code_blocks:
                                full_content += "\n\n" + "\n\n".join(code_blocks)

                            # Create Haystack Document
                            documents.append(Document(
                                content=full_content,
                                meta={"url": url, "title": title}
                            ))
                        except json.JSONDecodeError:
                            print(f"Error parsing content in {filename}")

    return documents


In [28]:
import os
import re
import ast
from haystack import Document
documents = []

def load_dict_docs(docs_dir):
    doc_count = 0

    """Load scraped documents from text files"""

    #documents = []
    for filename in os.listdir(docs_dir):
        if filename.endswith('.txt'):
            file_path = os.path.join(docs_dir, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                pages = content.split('=' * 80)

                for page in pages:
                    if not page.strip():
                        continue

                    # Extract metadata and content
                    url_match = re.search(r'url: (.*?)$', page, re.MULTILINE)
                    print(url_match)
                    title_match = re.search(r'title: (.*?)$', page, re.MULTILINE)
                    content_match = re.search(r'content: ({.*?})\n---' , page, re.MULTILINE | re.DOTALL)

                    if url_match and title_match and content_match:
                        url = url_match.group(1).strip()
                        title = title_match.group(1).strip()
                        content_str = content_match.group(1).strip()

                        try:
                            # Use ast.literal_eval instead of json.loads
                            content_dict = ast.literal_eval(content_str)

                            # Format content for better processing
                            text_content = "\n\n".join(content_dict.get('text', []))
                            code_blocks = []
                            for block in content_dict.get('code_blocks', []):
                                lang = block.get('language', '')
                                code = block.get('code', '')
                                code_blocks.append(f"```{lang}\n{code}\n```")

                            full_content = text_content
                            if code_blocks:
                                full_content += "\n\n" + "\n\n".join(code_blocks)

                            # Create Document directly
                            unique_id = f"{url}_{doc_count}"
                            doc_count += 1
                            documents.append(Document(
                                id=unique_id,
                                content=full_content,
                                meta={"url": url, "title": title}
                            ))
                            #print(f"Loaded document from {filename}")
                            #return documents
                        except (SyntaxError, ValueError) as e:
                            print(f"Error parsing content in {filename}: {e}")

    return documents


In [29]:
import os
import glob
docs_dir = os.path.join(os.getcwd(), '/content/sample_data/docs')
files = glob.glob(os.path.join(docs_dir, "fastapi_docs_batch_*.txt"))
print(files)
documents = load_dict_docs(docs_dir)
print (documents[1])

preprocessing_pipeline.run({
    "document_embedder": {"documents": documents}
})

print(f"Processed and stored {document_store.count_documents()} documents")

['/content/sample_data/docs/fastapi_docs_batch_8.txt', '/content/sample_data/docs/fastapi_docs_batch_11.txt', '/content/sample_data/docs/fastapi_docs_batch_3.txt', '/content/sample_data/docs/fastapi_docs_batch_13.txt', '/content/sample_data/docs/fastapi_docs_batch_10.txt', '/content/sample_data/docs/fastapi_docs_batch_15.txt', '/content/sample_data/docs/fastapi_docs_batch_7.txt', '/content/sample_data/docs/fastapi_docs_batch_5.txt', '/content/sample_data/docs/fastapi_docs_batch_1.txt', '/content/sample_data/docs/fastapi_docs_batch_9.txt', '/content/sample_data/docs/fastapi_docs_batch_6.txt', '/content/sample_data/docs/fastapi_docs_batch_4.txt', '/content/sample_data/docs/fastapi_docs_batch_2.txt', '/content/sample_data/docs/fastapi_docs_batch_14.txt', '/content/sample_data/docs/fastapi_docs_batch_12.txt']
<re.Match object; span=(0, 55), match='url: https://fastapi.tiangolo.com/virtual-environ>
<re.Match object; span=(2, 57), match='url: https://fastapi.tiangolo.com/tutorial/first->
<re

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Processed and stored 294 documents


In [43]:
from haystack.components.retrievers import InMemoryEmbeddingRetriever
from haystack.components.embedders import SentenceTransformersTextEmbedder
retriever = InMemoryEmbeddingRetriever(document_store=document_store)
query_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")



# Create simple pipeline
retrieval_pipeline = Pipeline()
retrieval_pipeline.add_component("query_embedder", query_embedder)
retrieval_pipeline.add_component("retriever", retriever)

retrieval_pipeline.connect("query_embedder", "retriever")



# Test some queries
test_queries = [
    "How to use FastAPI with path parameters? and small example please and make small comparison with django",
    "What are FastAPI dependencies? and small example please",
    "How to handle file uploads in FastAPI? and small example please"
]

for query in test_queries:
    print(f"\nQuery: {query}")
    results = retrieval_pipeline.run({
        "query_embedder": {"text": query},
        "retriever": {


            "top_k": 1  # Get top 2 most relevant documents
        }
    })
    response = client.models.generate_content(
    model="gemini-2.0-flash", contents=query+results["retriever"]["documents"][0].content
)
    print(response.text)
    print(results["retriever"]["documents"][0].meta)

    print("\nRetrieved Documents:")
    '''for doc in results["retriever"]["documents"]:
        print("-" * 50)
        print(f"Score: {doc.score:.4f}")
        #print(f"Content: {doc.content[:200]}...")
        #print(f"text: {doc.content }...")


        if isinstance(doc.content, dict):
          text_content = ' '.join(doc.content.get('text', []))
          code_blocks = doc.content.get('code_blocks', [])

          print("\nText Content:")
          print(text_content  + "...")

          if code_blocks:
            print("\nCode Examples:")
            for block in code_blocks:
                if block['code'].strip():
                    print(f"\nLanguage: {block['language']}")
                    print(block['code'] )
        else:
          print("\nContent:")
          #print(doc.content['text'] + "...")
          #print(doc.content['code_blocks'])
          print(doc.content[:300] + "...")'''


Query: How to use FastAPI with path parameters? and small example please and make small comparison with django


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

```python
from fastapi import FastAPI

app = FastAPI()

# Example 1: Basic path parameter
@app.get("/items/{item_id}")
async def read_item(item_id: int):
    return {"item_id": item_id}

# Example 2: Path parameter with type hint for validation
@app.get("/users/{user_id}")
async def read_user(user_id: str):
    return {"user_id": user_id}

# Example 3: Multiple path parameters
@app.get("/cities/{city_name}/users/{user_id}")
async def read_user_from_city(city_name: str, user_id: int):
    return {"city_name": city_name, "user_id": user_id}

# Example 4: Path parameter with enum
from enum import Enum
class ModelName(str, Enum):
    alexnet = "alexnet"
    resnet = "resnet"
    lenet = "lenet"

@app.get("/models/{model_name}")
async def get_model(model_name: ModelName):
    if model_name is ModelName.alexnet:
        return {"model_name": model_name, "message": "Deep Learning FTW!"}
    if model_name.value == "lenet":
        return {"model_name": model_name, "message": "LeCNN all the way

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Okay, let's break down FastAPI dependencies with a clear explanation and a focused example.

**What are FastAPI Dependencies?**

FastAPI dependencies are a powerful mechanism for code reuse, security, and modularity in your API. They allow you to inject reusable logic into your path operations (your API endpoints). This injected logic is executed *before* your path operation function, and the result of that logic can be passed as a parameter to your path operation.

**Key Concepts**

*   **Dependency Injection:** The core idea is that instead of your path operation function creating or fetching its own dependencies (e.g., database connections, user authentication), FastAPI "injects" them. This promotes loose coupling and testability.

*   **`Depends()`:**  The `Depends()` function is the primary tool for declaring dependencies.  You wrap a callable (usually a function) inside `Depends()`.  FastAPI will then call that callable and pass its return value to your path operation as an argum

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

```python
from typing import Annotated

from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
import shutil  # For saving files

app = FastAPI()


@app.post("/uploadfile/")
async def create_upload_file(file: UploadFile):
    try:
        # Option 1: Read the entire file content into memory (for small files)
        contents = await file.read()
        print(f"File size: {len(contents)} bytes")

        # Option 2: Save the file to disk (recommended for larger files)
        file_path = f"uploads/{file.filename}"  # Create an "uploads" directory if needed
        with open(file_path, "wb") as f:
            shutil.copyfileobj(file.file, f)  # Use shutil for efficient copying

        return JSONResponse(content={
            "filename": file.filename,
            "content_type": file.content_type,
            "file_saved_to": file_path  # Provide the path where the file was saved
        })
    except Exception as e:
        return JSONResponse(content

In [33]:
!uv pip install -q -U google-genai

In [36]:
from google import genai
from google.colab import userdata

client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))
#for queries
response = client.models.generate_content(
    model="gemini-2.0-flash", contents="Explain sachin greatness"
)
print(response.text)

Sachin Tendulkar is widely considered one of the greatest cricketers of all time, and for good reason. His greatness isn't just about numbers, though those are certainly impressive. It's a combination of talent, dedication, consistency, and the impact he had on the sport and the nation of India. Here's a breakdown of what made him so great:

**1. Sheer Talent and Technique:**

*   **Exceptional Hand-Eye Coordination:** Sachin possessed an uncanny ability to pick up the line and length of the ball with incredible speed and adjust his shot accordingly. This allowed him to play fast bowlers with confidence and spinners with finesse.
*   **Masterful Strokeplay:** He had a wide array of shots at his disposal, from powerful drives and cuts to delicate flicks and late cuts. He could adapt his game to different conditions and bowlers.
*   **Solid Technique:** He had a classical, orthodox batting technique that was both aesthetically pleasing and incredibly effective. This provided him with a s